### Imports

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

### Loading and viewing the data

In [ ]:
train_raw = pd.read_csv('./Data/training_set_values.csv')
test_raw = pd.read_csv('./Data/test_set_values.csv')

train_raw['train'] = 1
test_raw['train'] = 0
data = pd.concat([train_raw, test_raw])

data.head()

In [ ]:
# Overview

data.info()

In [ ]:
# Identify missing values in numerical data

int_var = ['population','gps_height','construction_year']
float_var = ['amount_tsh','longitude']

for var in int_var:
    print('{}:'.format(var))
    display(data[var].min())
    display(len(data[data[var] == 0]))

for var in float_var:
    print('{}:'.format(var))
    display(data[var].min())
    display(len(data[data[var] == 0.0]))

print('latitude:')
display(data['latitude'].min())
display(len(data[(data['latitude'] > -0.001) & (data['latitude'] < 0.001)]))

### Data preparation

In [ ]:
# Replace zeros by NaN

for var in int_var:
    data[var].replace(0, np.nan, inplace=True)
    
for var in float_var:
    data[var].replace(0.0, np.nan, inplace=True)

data['latitude'].where((data['latitude'] < -0.001) | (data['latitude'] > 0.001), other= np.nan, inplace=True,axis=0)

In [ ]:
# Logarithmic scaling of amount_tsh and population (optional)

data['amount_tsh']=data.apply(lambda row: np.log1p(row['amount_tsh']),axis=1)
data['population']=data.apply(lambda row: np.log1p(row['population']),axis=1)

train = data[data['train'] == 1]
test = data[data['train'] == 0]

In [ ]:
# Duplicate critical columnns for imputation based on normal distribution and random choice

null_features = ['longitude','latitude','gps_height','population','construction_year','amount_tsh']
#no calculations for num_private since they are dropped later (too many missing values)

for null_feature in null_features:
    data['_'.join([null_feature, 'imp_normal'])] = data[null_feature]
    data['_'.join([null_feature, 'imp_random_choice'])] = data[null_feature]

In [ ]:
# Drop redundant features and features that do not seem to have an impact (choose variant)

data.drop(['extraction_type_group','extraction_type_class','payment','quality_group','source_class','source_type','waterpoint_type_group','management_group','quantity_group','wpt_name','num_private','recorded_by'],axis=1,inplace=True)
#data.drop(['num_private','recorded_by'],axis=1,inplace=True)

In [ ]:
data.head()

### Imputation of numerical features by normal distribution

In [ ]:
# Add columns for mean and standard deviation of critical features based on 'region', 'ward' and 'overall'

divisions = ['region', 'ward']

for null_feature in null_features:
    data['_'.join([null_feature, 'mean', 'overall'])] = train[null_feature].mean()
    data['_'.join([null_feature, 'std', 'overall'])] = train[null_feature].std()
    for division in divisions:
        new_feature_name_mean = '_'.join([null_feature, 'mean', division])
        new_feature_name_std = '_'.join([null_feature, 'std', division])
        
        calcs_mean = train.groupby(division)[null_feature].mean()
        calcs_std = train.groupby(division)[null_feature].std()
        for value in train[division].unique() :
            data.loc[data[division]==value, new_feature_name_mean] = calcs_mean[value]
            data.loc[data[division]==value, new_feature_name_std] = calcs_std[value]

In [ ]:
# 1st step: Impute missing values with random numbers generated by normal distribution based on mean, std by 'ward'
# 2nd step (only applied on remaining null values): Impute missing values with random numbers generated by normal distribution based on mean, std by 'region'
# 3rd step (only applied on remaining null values): Impute missing values with random numbers generated by normal distribution based on mean, std by 'overall'

divisions_total = ['ward', 'region', 'overall']

for null_feature in null_features:
    for division in divisions_total:
        data['_'.join([null_feature,'imp_normal'])] = data.apply(lambda row: np.random.normal(loc=row['_'.join([null_feature,'mean',division])], scale=row['_'.join([null_feature,'std',division])]) if math.isnan(row['_'.join([null_feature,'imp_normal'])]) else row['_'.join([null_feature,'imp_normal'])], axis=1)
        display('Missing values after imputation in {} by {}: {}'.format(null_feature, division, data['_'.join([null_feature,'imp_normal'])].isnull().sum()))

### Imputation of numerical features by random choice

In [ ]:
# Add columns with list of values in corresponding group of 'region' and 'ward', respectively

for null_feature in null_features:
    overall_list = list(train[null_feature])
    overall_list = [x for x in overall_list if not math.isnan(x)]
    data['_'.join([null_feature, 'list', 'overall'])] = data.apply(lambda row: overall_list, axis=1)
    display(null_feature, 'overall list done')
    for division in divisions:
        feature_name = '_'.join([null_feature, 'list', division])
        lists = train.groupby(division)[null_feature].apply(list)
        data[feature_name] = data.apply(lambda row: list() if row[division] not in train[division].unique() else lists[row[division]], axis=1)
        data[feature_name] = data[feature_name].apply(lambda lst: [x for x in lst if not math.isnan(x)])
        data[feature_name] = data[feature_name].apply(lambda x: np.nan if not x else x)
        display(null_feature, division)

In [ ]:
# 1st step: Impute missing values with empirical distribution grouped by 'ward'
# 2nd step (only applied on remaining null values): Impute missing values with empirical distribution grouped by 'region'
# 3rd step (only applied on remaining null values): Impute missing values with empirical distribution grouped by 'overall'

for null_feature in null_features:
    for division in divisions_total:        
        data['_'.join([null_feature,'imp_random_choice'])] = data.apply(lambda row: row['_'.join([null_feature,'imp_random_choice'])] if not np.isnan(row['_'.join([null_feature,'imp_random_choice'])]).any() else (np.random.choice(a=row['_'.join([null_feature,'list',division])]) if not np.isnan(row['_'.join([null_feature,'list',division])]).any() else np.nan), axis=1)
        display('Missing values after imputation in {} by {}: {}'.format(null_feature, division, data['_'.join([null_feature,'imp_random_choice'])].isnull().sum()))

### Imputation of numerical features by mean/median

In [ ]:
# Add columns for median of critical integer features based on region, ward, overall

float_var.append('latitude')

for var in int_var:
    data['_'.join([var, 'median', 'overall'])] = train[var].median()
    for division in divisions:
        new_feature_name_median = '_'.join([var, 'median', division])
        calcs_median = train.groupby(division)[var].median()
        for value in train[division].unique() :
            data.loc[data[division]==value, new_feature_name_median] = calcs_median[value]

In [ ]:
# 1st step: Impute missing values with mean and median by 'ward'
# 2nd step (only applied on remaining null values): Impute missing values with mean and median by 'region'
# 3rd step (only applied on remaining null values): Impute missing values with overall mean and median

for var in float_var:
    for division in divisions_total:
        data[var] = data.apply(lambda row: row['_'.join([var,'mean',division])] if math.isnan(row[var]) else row[var], axis=1)
        display('Missing values after imputation in {} by {}: {}'.format(var, division, data[var].isnull().sum()))

for var in int_var:
    for division in divisions_total:
        data[var] = data.apply(lambda row: row['_'.join([var,'median',division])] if math.isnan(row[var]) else row[var], axis=1)
        display('Missing values after imputation in {} by {}: {}'.format(var, division, data[var].isnull().sum()))  

In [ ]:
# Drop columns used for imputation and generation of random numbers

drop_columns = list()
measures = 'mean std list'.split()
for null_feature in null_features:
    for division in divisions_total:
        for measure in measures:
            drop_columns.append('_'.join([null_feature, measure, division]))
            
for var in int_var:
    for division in divisions_total:
        for measure in measures:
            drop_columns.append('_'.join([var, 'median', division]))
            
data.drop(columns=drop_columns, inplace=True)

### Feature generation

In [ ]:
# Create new feature that gives information about operational time
data['date_recorded'] = pd.to_datetime(data['date_recorded_x'])
data['operation_years'] = data.date_recorded.dt.year - data.construction_year
data['operation_years'] = data['operation_years'].astype(int)

methods = ['imp_normal','imp_random_choice']
for method in methods:
    data['_'.join(['operation_years',method])] = data.date_recorded.dt.year - data['_'.join(['construction_year',method])]
    data['_'.join(['operation_years',method])] = data['_'.join(['operation_years',method])].astype(int)
data.drop(['date_recorded'],axis=1,inplace=True)

data.head()

### Scaling of numerical features

No scaling of numerical features for data version 0.1

In [ ]:
# Scale numerical features
'''
num_features=['latitude','longitude','operation_years','amount_tsh', 'gps_height', 'population']
scaler = MinMaxScaler()

for s in split:
    s[num_features] = scaler.fit_transform(s[num_features])
'''

### Preparation of categorical features

No grouping and factorizing of categorical features in data version 0.1; missing values replaced by 'Missing' only

In [ ]:
# Fill missing values in categorical features by 'Missing'

cat_features = 'funder installer subvillage public_meeting scheme_management scheme_name permit'.split()

for feature in cat_features:
    data[feature].replace(np.nan, 'Missing', inplace=True)

In [ ]:
'''data['funder'].value_counts()
data['installer'].value_counts()
data['scheme_name'].value_counts()'''

In [ ]:
# Reduce dimension of categorical variables

'''# 'funder'
data = data.assign(count = data.groupby('funder')['funder'].transform('count'))\
.sort_values(by = ['count','funder'], ascending = [False,True])

data.loc[data['count'] < 1050, 'funder'] = 'Others'
data['funder'].replace(np.nan, 'Others', inplace=True)
del data['count']

# 'installer'
data = data.assign(count = data.groupby('installer')['installer'].transform('count'))\
.sort_values(by = ['count','installer'], ascending = [False,True])

data.loc[data['count'] < 765, 'installer'] = 'Others'
data.loc[data['installer'] == '0', 'installer'] = 'Others'
data['installer'].replace(np.nan, 'Others', inplace=True)
del data['count']

# 'scheme_name'
data = data.assign(count = data.groupby('scheme_name')['scheme_name'].transform('count'))\
.sort_values(by = ['count','scheme_name'], ascending = [False,True])

data.loc[data['count'] < 296, 'scheme_name'] = 'Others'
data.loc[data['scheme_name'] == '0', 'scheme_name'] = 'Others'
data['scheme_name'].replace(np.nan, 'Others', inplace=True)
del data['count']'''

In [ ]:
# Factorize features for evaluations

data['funder'] = pd.factorize(data['funder'])[0]
data['installer'] = pd.factorize(data['installer'])[0]
data['basin'] = pd.factorize(data['basin'])[0]
data['subvillage'] = pd.factorize(data['subvillage'])[0]
data['region'] = pd.factorize(data['region'])[0]
data['lga'] = pd.factorize(data['lga'])[0]
data['ward'] = pd.factorize(data['ward'])[0]
data['scheme_management'] = pd.factorize(data['scheme_management'])[0]
data['scheme_name'] = pd.factorize(data['scheme_name'])[0]
data['extraction_type'] = pd.factorize(data['extraction_type'])[0]
data['management'] = pd.factorize(data['management'])[0]
data['payment_type'] = pd.factorize(data['payment_type'])[0]
data['water_quality'] = pd.factorize(data['water_quality'])[0]
data['quantity'] = pd.factorize(data['quantity'])[0]
data['waterpoint_type'] = pd.factorize(data['waterpoint_type'])[0]
data['permit'] = pd.factorize(data['permit'])[0]
data['source'] = pd.factorize(data['source'])[0]'''

### Split data into train and test data set

In [ ]:
train_df = data[data["train"] == 1]
test_df = data[data["train"] == 0]

train_df.drop(["train"], axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

### Save data to csv files

In [ ]:
pd.DataFrame(train_df).to_csv("./Data/train_cleaned_v0.2.csv", index=False)
pd.DataFrame(test_df).to_csv("./Data/test_cleaned_v0.2.csv", index=False)